### 参照 https://github.com/zergtant/pytorch-handbook/blob/master/chapter3/3.2-mnist.ipynb

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
torch.__version__

'1.0.0'

In [3]:
BATCH_SIZE=512 #大概需要2G的显存
EPOCHS=10 # 总共训练批次
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 让torch判断是否使用GPU，建议使用GPU环境，因为会快很多

In [4]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data/mnist', train=True, download=False,   ## D:\360downloads\毕业论文\pytorch\data
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [5]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data/mnist', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

In [6]:
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        # batch*1*28*28（每次会送入batch个样本，输入通道数1（黑白图像），图像分辨率是28x28）
        # 下面的卷积层Conv2d的第一个参数指输入通道数，第二个参数指输出通道数，第三个参数指卷积核的大小
        self.conv1 = nn.Conv2d(1, 10, 5) # 输入通道数1，输出通道数10，核的大小5
        self.conv2 = nn.Conv2d(10, 20, 3) # 输入通道数10，输出通道数20，核的大小3
        # 下面的全连接层Linear的第一个参数指输入通道数，第二个参数指输出通道数
        self.fc1 = nn.Linear(20*10*10, 500) # 输入通道数是2000，输出通道数是500
        self.fc2 = nn.Linear(500, 10) # 输入通道数是500，输出通道数是10，即10分类
    def forward(self,x):
        in_size = x.size(0) # 在本例中in_size=512，也就是BATCH_SIZE的值。输入的x可以看成是512*1*28*28的张量。
        out = self.conv1(x) # batch*1*28*28 -> batch*10*24*24（28x28的图像经过一次核为5x5的卷积，输出变为24x24）
        out = F.relu(out) # batch*10*24*24（激活函数ReLU不改变形状））
        out = F.max_pool2d(out, 2, 2) # batch*10*24*24 -> batch*10*12*12（2*2的池化层会减半）
        out = self.conv2(out) # batch*10*12*12 -> batch*20*10*10（再卷积一次，核的大小是3）
        out = F.relu(out) # batch*20*10*10
        out = out.view(in_size, -1) # batch*20*10*10 -> batch*2000（out的第二维是-1，说明是自动推算，本例中第二维是20*10*10）
        out = self.fc1(out) # batch*2000 -> batch*500
        out = F.relu(out) # batch*500
        out = self.fc2(out) # batch*500 -> batch*10
        out = F.log_softmax(out, dim=1) # 计算log(softmax(x))
        return out

In [7]:
model = ConvNet().to(DEVICE)
optimizer = optim.Adam(model.parameters())

In [8]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if(batch_idx+1)%30 == 0: 
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [9]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:
for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)

Train Epoch: 1 [14848/60000 (25%)]	Loss: 0.317391
Train Epoch: 1 [30208/60000 (50%)]	Loss: 0.191830
Train Epoch: 1 [45568/60000 (75%)]	Loss: 0.105422

Test set: Average loss: 0.0932, Accuracy: 9725/10000 (97%)

Train Epoch: 2 [14848/60000 (25%)]	Loss: 0.083484
Train Epoch: 2 [30208/60000 (50%)]	Loss: 0.065168
Train Epoch: 2 [45568/60000 (75%)]	Loss: 0.049209

Test set: Average loss: 0.0547, Accuracy: 9831/10000 (98%)

Train Epoch: 3 [14848/60000 (25%)]	Loss: 0.048821
Train Epoch: 3 [30208/60000 (50%)]	Loss: 0.032522
Train Epoch: 3 [45568/60000 (75%)]	Loss: 0.049587

Test set: Average loss: 0.0434, Accuracy: 9858/10000 (99%)

Train Epoch: 4 [14848/60000 (25%)]	Loss: 0.022621
Train Epoch: 4 [30208/60000 (50%)]	Loss: 0.058208
Train Epoch: 4 [45568/60000 (75%)]	Loss: 0.034728

Test set: Average loss: 0.0405, Accuracy: 9870/10000 (99%)

Train Epoch: 5 [14848/60000 (25%)]	Loss: 0.026964
Train Epoch: 5 [30208/60000 (50%)]	Loss: 0.037705
Train Epoch: 5 [45568/60000 (75%)]	Loss: 0.038580

Test 

In [16]:
i = 1
for data, target in test_loader:
    data, target = data.to(DEVICE), target.to(DEVICE)
    output = model(data)
    print('pre:',output.argmax(dim = 1))
    print('tar:',target)
    i = i + 1 
    if(i == 10):
        break

pre: tensor([8, 9, 3, 2, 1, 9, 3, 1, 4, 2, 9, 7, 0, 1, 9, 5, 0, 5, 0, 2, 3, 2, 3, 1,
        5, 2, 7, 5, 8, 4, 2, 0, 6, 3, 9, 0, 4, 4, 2, 6, 2, 2, 7, 9, 8, 9, 0, 2,
        1, 2, 4, 3, 5, 9, 2, 6, 9, 6, 1, 9, 7, 8, 6, 0, 7, 8, 3, 2, 0, 4, 0, 5,
        2, 5, 4, 2, 8, 4, 8, 8, 7, 0, 9, 4, 7, 2, 3, 5, 5, 9, 8, 3, 1, 4, 3, 8,
        0, 0, 1, 9, 2, 8, 0, 6, 9, 1, 3, 3, 7, 7, 0, 3, 7, 6, 8, 3, 4, 8, 5, 7,
        1, 9, 2, 0, 3, 2, 0, 3, 9, 2, 3, 4, 0, 1, 5, 1, 4, 6, 3, 1, 0, 4, 1, 7,
        0, 7, 0, 7, 4, 9, 0, 4, 1, 1, 4, 0, 9, 2, 6, 4, 9, 0, 8, 2, 8, 7, 6, 3,
        5, 5, 0, 3, 9, 2, 7, 3, 0, 1, 0, 8, 1, 1, 0, 8, 7, 8, 2, 7, 1, 5, 2, 1,
        1, 3, 0, 3, 6, 8, 9, 7, 8, 6, 3, 4, 4, 1, 9, 9, 2, 1, 4, 1, 1, 0, 9, 2,
        3, 2, 0, 6, 1, 9, 3, 2, 1, 2, 0, 6, 0, 9, 1, 7, 7, 4, 9, 0, 4, 6, 8, 7,
        1, 9, 6, 3, 2, 1, 1, 0, 2, 3, 0, 5, 5, 7, 6, 2, 1, 3, 0, 1, 7, 4, 9, 1,
        1, 8, 6, 6, 3, 4, 8, 0, 3, 6, 5, 3, 2, 9, 4, 0, 4, 7, 3, 2, 0, 4, 6, 2,
        9, 4, 3, 1, 1, 0, 1, 7, 1, 

tar: tensor([8, 2, 9, 8, 8, 0, 3, 0, 6, 7, 7, 8, 0, 2, 1, 1, 6, 0, 6, 6, 6, 8, 6, 3,
        3, 8, 2, 8, 1, 8, 0, 1, 0, 4, 0, 8, 3, 9, 2, 5, 5, 6, 8, 8, 8, 2, 1, 3,
        3, 8, 2, 5, 3, 3, 3, 2, 7, 3, 3, 2, 2, 8, 3, 9, 1, 2, 6, 4, 9, 6, 3, 8,
        7, 6, 9, 9, 7, 1, 5, 1, 2, 8, 3, 3, 7, 3, 9, 5, 8, 7, 7, 9, 9, 2, 9, 8,
        0, 8, 1, 8, 6, 4, 7, 9, 4, 7, 7, 9, 2, 7, 1, 4, 7, 1, 3, 9, 3, 0, 7, 1,
        4, 6, 4, 7, 4, 7, 4, 8, 6, 7, 7, 0, 6, 9, 6, 8, 6, 4, 9, 8, 6, 8, 2, 1,
        6, 8, 9, 3, 2, 8, 5, 4, 4, 9, 1, 1, 6, 9, 5, 9, 9, 6, 2, 7, 6, 8, 4, 8,
        2, 2, 0, 0, 1, 5, 9, 2, 0, 1, 1, 2, 9, 1, 4, 0, 2, 9, 3, 8, 7, 6, 4, 2,
        6, 4, 5, 6, 7, 7, 8, 3, 0, 4, 4, 8, 1, 9, 7, 3, 1, 1, 7, 6, 9, 9, 5, 1,
        0, 4, 5, 5, 8, 5, 0, 4, 3, 7, 8, 6, 1, 6, 0, 8, 6, 7, 7, 6, 6, 6, 3, 1,
        9, 2, 9, 1, 3, 4, 4, 9, 1, 1, 0, 7, 6, 9, 6, 1, 9, 5, 5, 7, 9, 8, 8, 3,
        6, 4, 6, 5, 4, 9, 9, 6, 0, 0, 5, 1, 5, 8, 7, 4, 0, 1, 4, 0, 2, 3, 1, 4,
        8, 8, 0, 2, 9, 3, 2, 9, 5, 

        4, 0, 4, 4, 7, 2, 8, 5], device='cuda:0')
pre: tensor([4, 9, 1, 7, 5, 6, 8, 4, 5, 1, 2, 4, 2, 7, 9, 8, 6, 4, 3, 4, 1, 5, 4, 6,
        6, 1, 5, 4, 8, 1, 4, 2, 6, 6, 4, 0, 0, 6, 9, 5, 2, 3, 9, 8, 8, 3, 1, 1,
        4, 1, 1, 4, 3, 8, 8, 7, 4, 5, 6, 7, 8, 0, 2, 9, 6, 3, 2, 6, 7, 8, 0, 6,
        1, 3, 5, 9, 0, 7, 7, 4, 6, 2, 1, 7, 9, 8, 4, 7, 6, 8, 8, 9, 2, 5, 4, 0,
        1, 3, 2, 2, 8, 0, 1, 1, 7, 2, 0, 6, 2, 9, 8, 1, 0, 1, 8, 5, 7, 1, 1, 6,
        5, 6, 9, 8, 5, 1, 2, 3, 1, 3, 1, 7, 1, 1, 2, 3, 1, 0, 5, 7, 2, 2, 0, 0,
        1, 1, 9, 6, 2, 3, 6, 5, 3, 5, 2, 0, 6, 7, 3, 3, 0, 4, 7, 0, 6, 9, 3, 2,
        9, 3, 4, 1, 6, 6, 8, 7, 6, 0, 0, 2, 8, 0, 4, 3, 6, 1, 4, 1, 8, 3, 5, 5,
        8, 2, 0, 5, 5, 0, 7, 2, 1, 8, 2, 7, 5, 5, 8, 3, 6, 9, 9, 3, 8, 1, 7, 2,
        3, 2, 7, 0, 2, 3, 0, 6, 6, 8, 1, 5, 5, 2, 1, 1, 9, 3, 4, 5, 9, 9, 8, 9,
        6, 1, 4, 4, 6, 0, 0, 7, 6, 7, 2, 6, 0, 4, 3, 7, 3, 6, 7, 9, 1, 9, 5, 5,
        2, 2, 3, 9, 8, 8, 6, 9, 7, 5, 5, 9, 8, 5, 4, 2, 5, 7, 6, 

        5, 9, 6, 9, 2, 2, 9, 6], device='cuda:0')
pre: tensor([1, 5, 1, 0, 5, 0, 9, 3, 1, 1, 7, 2, 7, 4, 5, 1, 3, 5, 8, 6, 4, 9, 0, 6,
        1, 3, 1, 9, 0, 3, 0, 5, 0, 1, 4, 8, 8, 3, 2, 0, 6, 8, 3, 8, 4, 6, 4, 3,
        3, 2, 9, 4, 5, 9, 7, 3, 9, 1, 7, 5, 9, 8, 1, 2, 5, 1, 2, 8, 3, 8, 6, 8,
        3, 1, 2, 7, 4, 1, 8, 6, 9, 1, 0, 3, 4, 7, 5, 4, 3, 5, 6, 7, 0, 2, 1, 8,
        8, 4, 5, 7, 6, 5, 6, 8, 9, 1, 8, 8, 8, 2, 4, 0, 2, 9, 6, 0, 5, 4, 3, 5,
        6, 7, 6, 2, 8, 9, 1, 1, 3, 5, 8, 8, 2, 9, 8, 3, 5, 3, 5, 4, 3, 5, 8, 0,
        5, 4, 7, 6, 7, 2, 7, 4, 8, 1, 8, 7, 6, 1, 6, 0, 0, 9, 9, 1, 3, 8, 1, 6,
        4, 4, 4, 3, 5, 1, 9, 0, 8, 1, 4, 7, 3, 8, 2, 2, 5, 7, 3, 4, 1, 3, 0, 4,
        7, 2, 1, 1, 6, 6, 2, 0, 8, 6, 0, 7, 7, 4, 9, 3, 1, 5, 7, 0, 2, 9, 2, 3,
        8, 5, 7, 4, 5, 2, 8, 3, 4, 6, 5, 9, 6, 2, 4, 2, 7, 4, 1, 0, 8, 0, 4, 9,
        8, 9, 6, 2, 7, 0, 0, 2, 2, 9, 6, 6, 3, 6, 7, 0, 4, 7, 7, 3, 4, 3, 3, 8,
        5, 7, 6, 3, 1, 8, 9, 2, 3, 8, 4, 6, 3, 5, 3, 9, 2, 4, 1, 

In [17]:
# 测试bz对测试的影响（我这里是把bz当做检测时输入的数量）
test_loader_bz1 = torch.utils.data.DataLoader(
        datasets.MNIST('data/mnist', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=1, shuffle=True)
i = 1
for data, target in test_loader_bz1:
    data, target = data.to(DEVICE), target.to(DEVICE)
    output = model(data)
    print('pre:',output.argmax(dim = 1))
    print('tar:',target)
    i = i + 1 
    if(i == 10):
        break

pre: tensor([3], device='cuda:0')
tar: tensor([3], device='cuda:0')
pre: tensor([8], device='cuda:0')
tar: tensor([8], device='cuda:0')
pre: tensor([2], device='cuda:0')
tar: tensor([2], device='cuda:0')
pre: tensor([0], device='cuda:0')
tar: tensor([0], device='cuda:0')
pre: tensor([4], device='cuda:0')
tar: tensor([4], device='cuda:0')
pre: tensor([7], device='cuda:0')
tar: tensor([7], device='cuda:0')
pre: tensor([8], device='cuda:0')
tar: tensor([8], device='cuda:0')
pre: tensor([1], device='cuda:0')
tar: tensor([1], device='cuda:0')
pre: tensor([7], device='cuda:0')
tar: tensor([7], device='cuda:0')
